In [9]:
#Model imports
import tensorflow as tf
#import pandas as pd
#import plotly.graph_objects as go
import numpy

In [12]:
tf.config.list_physical_devices('GPU')


[]

In [95]:
# load data
data = pd.read_csv("training-dataset.csv")
data.drop(['datetime','datetime.1','3570'], axis=1,inplace=True)
data.dtypes

3307          float64
3616          float64
3576          float64
4031_level    float64
4046_level    float64
4058_level    float64
4122_level    float64
4008_level    float64
Day sin       float64
Day cos       float64
Year sin      float64
Year cos      float64
dtype: object

In [96]:
#split test train data
train_data = data.iloc[:int(0.8*len(data))]
test_data = data.iloc[int(0.8*len(data)):]


In [97]:
# extract input and output columns
#assumes there are 6 columns of input values with labels in the 7th column

X_train = train_data.iloc[:,:6]
y_train = train_data.iloc[:,6]
X_test = test_data.iloc[:,:6]
y_test = test_data.iloc[:,6]

In [98]:
# create LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [99]:
# plot the model
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [100]:
# reshape data for LSTM
X_train_np = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_np = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)


In [101]:
X_train = X_train_np.astype(numpy.float32)

In [102]:
# train model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Epoch 1/50
1102/1102 [==============================] - 6s 4ms/step - loss: 0.0340 - val_loss: 0.1550
Epoch 2/50
1102/1102 [==============================] - 4s 4ms/step - loss: 0.0333 - val_loss: 0.1627
Epoch 3/50
1102/1102 [==============================] - 5s 4ms/step - loss: 0.0333 - val_loss: 0.1566
Epoch 4/50
1102/1102 [==============================] - 5s 4ms/step - loss: 0.0332 - val_loss: 0.1533
Epoch 5/50
1102/1102 [==============================] - 5s 5ms/step - loss: 0.0329 - val_loss: 0.1656
Epoch 6/50
1102/1102 [==============================] - 5s 4ms/step - loss: 0.0324 - val_loss: 0.1697
Epoch 7/50
1102/1102 [==============================] - 6s 5ms/step - loss: 0.0321 - val_loss: 0.1744
Epoch 8/50
1102/1102 [==============================] - 5s 4ms/step - loss: 0.0320 - val_loss: 0.1794
Epoch 9/50
1102/1102 [==============================] - 5s 4ms/step - loss: 0.0320 - val_loss: 0.1762
Epoch 10/50
1102/1102 [==============================] - 4s 4ms/step - loss: 0.031

KeyboardInterrupt: 

In [ ]:
# make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)


276/276 [==============================] - 1s 2ms/step


In [ ]:
#save model to file

In [ ]:
# plot train and test predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(train_predictions))), y=train_predictions, name='Train'))
fig.add_trace(go.Scatter(x=list(range(len(test_predictions))), y=test_predictions, name='Test'))
fig.show()

In [ ]:

# to determine the most important input features using the SHAP library

import shap

# load the model and the data
model = model
#X_train = pd.read_csv("X_train.csv").values

# explain the model's predictions using SHAP values
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# summarize the importance of each feature
shap.summary_plot(shap_values, feature_names=X_test.columns)

TypeError: 'numpy.ndarray' object is not callable